In [2]:
import os
import sys
import pathlib
import typing

import numpy as np

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D
from keras.layers import Dropout, BatchNormalization
from keras.callbacks import Callback
from keras.optimizers import adam, SGD

import ml.generators.mip_generator as generator

import tensorflow as tf

In [5]:
num_filts, filt_dims = 96, 3
channels, im_dims = 3, 120
output_dim = 1

epochs = 20
batch_size = 16

In [9]:
TrainGen = generator
ValGen = generator
train_gen = TrainGen(dims=(channels, im_dims, im_dims),
                          batch_size=batch_size,
                          augment_data=True,
                          extend_dims=False,
                          validation=False)
val_gen = ValGen(dims=(channels, im_dims, im_dims),
                          batch_size=batch_size,
                          augment_data=True,
                          extend_dims=False,
                          validation=True)

TypeError: 'module' object is not callable

In [42]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [33]:
model = Sequential()
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first',
                 activation='relu', padding='same', input_shape=(channels, im_dims, im_dims)))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same', strides=(2,2)))
model.add(BatchNormalization())
num_filts *= 2
model.add(Dropout(0.4))
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same', strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (1, 1), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(1, (1, 1), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D(data_format='channels_first'))
model.add(Dense(output_dim, activation='sigmoid'))
model.compile(optimizer=adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 192, 120, 120)     5376      
_________________________________________________________________
batch_normalization_19 (Batc (None, 192, 120, 120)     480       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 192, 120, 120)     331968    
_________________________________________________________________
batch_normalization_20 (Batc (None, 192, 120, 120)     480       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 192, 60, 60)       331968    
_________________________________________________________________
batch_normalization_21 (Batc (None, 192, 60, 60)       240       
_________________________________________________________________
dropout_5 (Dropout)          (None, 192, 60, 60)       0         
__________

In [ ]:
mc_callback = ModelCheckpoint(filepath='tmp/all_conv_weights.hdf5', verbose=1)
model.fit_generator(
    generator=training_gen.generate(),
    steps_per_epoch=training_gen.get_steps_per_epoch(),
    validation_data=validation_gen.generate(),
    validation_steps=validation_gen.get_steps_per_epoch(),
    epochs=epochs,
    callbacks=[mc_callback],
    verbose=1,
    max_queue_size=1)